In [1]:
#Import Packages
import os
import glob
import time

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from IPython.display import display, HTML

from langdetect.lang_detect_exception import LangDetectException
from langdetect import detect, DetectorFactory
import spacy

# A. Combine Playlist CSVs into combined_df

The playlist csv files were gathered using a website that takes a Spotify playlist link and returns a file with all of the songs information / metadata. 17 different public playlists from a myriad of genres and time periods were selected as the basis for this dataset. Upon running these playlists through the website and concatenating the resulting csv files, we were left with a dataset of nearly 9,000 songs that is the basis for the beginning of this notebook.

In [2]:
# Set Working Directory
working_dir = r"C:\Users\Ryan\OneDrive\Documents\Final Project\Final_Script\Data"
os.chdir(working_dir)
# Our first dataset consists of a number of popular Spotify Playlists downloaded onto CSV Files
# Set the folder path where your CSVs are
folder_path = "Playlist"

# Use glob to get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and combine them into one DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

combined_df

,#,Song,Artist,Popularity,BPM,Genres,Parent Genres,Album,Album Date,Time,...,Live,Loud (Db),Key,Time Signature,Added At,Spotify Track Id,Album Label,Camelot,ISRC,Time
0,1,Murder On The Dancefloor,Sophie Ellis-Bextor,2,117,NaN,NaN,Read My Lips,2002-01-01,03:50,...,30,-5,C♯/D♭ minor,4,2024-02-14,0ndXb2CwhrqtyHK6o6eVNd,Polydor Records,12A,GBAKW0100229,NaN
1,2,Dog Days Are Over,Florence + The Machine,83,150,baroque pop,"Classical, Pop",Lungs (Deluxe Version),2009-00-00,04:12,...,10,-5,G,4,2024-02-14,456WNXWhDwYOSf5SpTuqxd,Universal-Island Records Ltd.,9B,GBUM70900209,NaN
2,3,Say It Right,Nelly Furtado,84,117,NaN,NaN,Loose,2006-01-01,03:43,...,0,-6,C♯/D♭,4,2024-02-14,2aI21FnmY7TJVKeMaoQZ0t,Mosley / Geffen,3B,USUM70603368,NaN
3,4,Where Is The Love?,Black Eyed Peas,3,94,NaN,NaN,Elephunk,2003-00-00,04:32,...,10,-3,F,4,2024-02-14,3CNqo3gYrfexdrtjFmC9he,A&M,7B,USIR10311862,NaN
4,5,Walking On A Dream,Empire Of The Sun,87,127,NaN,NaN,Walking On A Dream (10th Anniversary Edition),2008-10-03,03:18,...,0,-6,F minor,4,2024-02-14,5r5cp9IpziiIsR6b93vcnQ,EMI Recorded Music Australia Pty Ltd,4A,AUEI10800039,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8681,334,San Tropez,Pink Floyd,59,122,"progressive rock, psychedelic rock, classic ro...",Rock,Meddle,1971-11-11,03:43,...,0,-15,C,4,2024-10-17,3kbUFmLV5SHVQonfOpNgkd,Pink Floyd Records,8B,GBN9Y1100063,NaN
8682,335,Wish You Were Here,Pink Floyd,83,123,"progressive rock, psychedelic rock, classic ro...",Rock,Wish You Were Here,1975-09-12,05:34,...,80,-16,G,4,2024-12-04,6mFkJmJqdDVQ1REhVfGgd1,Pink Floyd Records,9B,GBN9Y1100088,NaN
8683,336,Mrs. Robinson - From The Graduate Soundtrack,Simon & Garfunkel,77,92,"folk rock, folk","Folk, Rock",Bookends,1968-04-03,04:04,...,0,-14,F#/G♭ minor,4,2024-12-04,0iOZM63lendWRTTeKhZBSC,Columbia,11A,USSM16800379,NaN
8684,337,Subterranean Homesick Blues,Bob Dylan,64,174,"folk rock, folk, singer-songwriter, roots rock...","Country, Rock, Folk",Bringing It All Back Home,1965-03-22,02:21,...,30,-10,E minor,4,2024-12-30,6k9DUKMJpWvu6eFG3O64Lg,Columbia,9A,USSM16501028,NaN


In [3]:
#List of the columns i combined_df
combined_df.columns.tolist()

['#',
 'Song',
 'Artist',
 'Popularity',
 'BPM',
 'Genres',
 'Parent Genres',
 'Album',
 'Album Date',
 'Time',
 'Dance',
 'Energy',
 'Acoustic',
 'Instrumental',
 'Happy',
 'Speech',
 'Live',
 'Loud (Db)',
 'Key',
 'Time Signature',
 'Added At',
 'Spotify Track Id',
 'Album Label',
 'Camelot',
 'ISRC',
 'Time ']

# B. Initial Clean the Dataset (remove duplicates, remixes, unnecessary columns, keep only the first listed artist, and split each song title at '-'¶

The clean_artist function standardizes artist names by splitting them at commas and keeping only the first listed artist. This helps isolate the primary artist when multiple collaborators are listed. However, some artists—like “Tyler, The Creator” and “Earth, Wind & Fire”—have commas in their actual names. To avoid incorrectly splitting these, the function checks for exceptions and preserves those names in full. Example: "Post Malone, Swae Lee" --> "Post Malone" (split at comma)

Songs are also split at "-" in their titles. This helps filter out phrases such as features.

In [4]:
# Define columns to drop
columns_to_drop = [
    "Added At", "Spotify Track Id", "Album Label", "#", "ISRC",
    "Album Date", "Time ", "Album"
]

# Define artist exceptions - these artists have commas in their names, but we don't want to split them
exceptions = ["Tyler, The Creator", "Earth, Wind & Fire"]

# Artist cleaning function - standardizes artist names (keeps exceptions intact)
# For non-exceptions: grabs the first artist listed before a comma (the main artist)
def clean_artist(artist_name):
    if artist_name in exceptions:                
        return artist_name.strip()
    return artist_name.split(",")[0].strip()

# Build final cleaned DataFrame step-by-step
clean_df = (
    combined_df
    .drop_duplicates(subset=["Song", "Artist"], keep="first")                      # Removes duplicates based on Song + Artist
    .query("~Song.str.contains('remix', case=False, na=False)", engine="python")   # Filters out song titles that contain the word "remix"   
    .drop(columns=[col for col in columns_to_drop if col in combined_df.columns])  # Drops columns defined in 'columns_to_drop'
    .assign(
        Artist_Clean=lambda df: df["Artist"].apply(clean_artist),                  # applies clean_artist functiom                
        Song_Clean=lambda df: df["Song"].str.split(" - ").str[0]                   # Splits each song title at '-' (cleans features, - live, etc.)
    )
    .reset_index(drop=True)
)

In [5]:
# Check to make sure the Artists in the exception list were preserved before saving
tyler_songs = clean_df[clean_df["Artist_Clean"] == "Tyler, The Creator"]
print(tyler_songs)

                        Song              Artist  Popularity  BPM Genres  \
4594  GONE, GONE / THANK YOU  Tyler, The Creator          79   79    NaN   
4624                 Glitter  Tyler, The Creator          70   79    NaN   
4840   ARE WE STILL FRIENDS?  Tyler, The Creator          86  187    NaN   
4946                DOGTOOTH  Tyler, The Creator          12   78    NaN   
5311              LUMBERJACK  Tyler, The Creator          66  171    NaN   
5412                 I THINK  Tyler, The Creator          77  121    NaN   
5416               EARFQUAKE  Tyler, The Creator          84   80    NaN   
5485     RUNNING OUT OF TIME  Tyler, The Creator          79   84    NaN   
5583          NEW MAGIC WAND  Tyler, The Creator          82  140    NaN   
5586           BEST INTEREST  Tyler, The Creator          83   98    NaN   
5838                    Noid  Tyler, The Creator          75   82    NaN   

     Parent Genres   Time  Dance  Energy  Acoustic  Instrumental  Happy  \
4594        

In [6]:
clean_df.to_csv("initial_songs_master.csv", index=False)

# C. Genre Scrape and Cleaning
Due to the long runtime of the genre scraping process, we've included a pre-scraped CSV (songs_with_genre_tags_full.csv) for direct import. This ensures faster execution and reproducibility.

The source code used for scraping genres is provided below, but commented out for convenience.



In [7]:
# Load the pre-scraped genre CSV file created from "initial_songs_master.CSV"
genre_df = pd.read_csv("songs_with_genre_tags_full.csv") 

# Get value counts for the 'Tag Source' column
# Track - genre scraped directly from the song
# Artist - inferred from the artist
# None - no genre tag was found
tag_source_counts = genre_df["Tag Source"].value_counts()

# Display the result
print(tag_source_counts)

Tag Source
track     5468
none       687
artist     505
Name: count, dtype: int64


In [8]:
# Keep only rows where Tag Source is 'track' - more reliable than artist or no tags. 
df_track_only = genre_df[genre_df["Tag Source"] == "track"]

# Optionally reset the index
df_track_only = df_track_only.reset_index(drop=True)

df_track_only

,Song,Artist,Genre Tags,Tag Source
0,Sweet Dreams (Are Made of This) - 2005 Remaster,"Eurythmics,Annie Lennox,Dave Stewart","80s, pop, new wave, female vocalists, synth pop",track
1,Smalltown Boy,Bronski Beat,"80s, new wave, synthpop, pop, synth pop",track
2,I'm Still Standing,Elton John,"pop, 80s, elton john, rock, classic rock",track
3,Funky Town,Lipps Inc.,"Disco, 80s, pop, 70s, dance",track
4,I'm So Excited,The Pointer Sisters,"80s, Disco, pop, dance, soul",track
...,...,...,...,...
5463,"You're The First, The Last, My Everything - Edit",Barry White,"soul, barry white, Disco, 70s, Love",track
5464,Get It On,T. Rex,"glam rock, 70s, classic rock, rock, glam",track
5465,We Are Family - 1995 Remaster,Sister Sledge,"Disco, 70s, dance, funk, soul",track
5466,Love Really Hurts Without You,Billy Ocean,"80s, pop, soul, 70s, Disco",track


In [9]:
# Save to a new CSV
df_track_only.to_csv("songs_genre_clean.csv", index=False)

# Lyric Scrape + Cleaning
Due to the long runtime of the lyric scraping process, we've included a pre-scraped CSV (week_2_songs_genre_lyrics.csv) for direct import. This CSV was built off "songs_genre_clean.CSV). This ensures faster execution and reproducibility.

The source code used for scraping lyrics is provided below, but commented out for convenience.

Removing rows with missing or empty lyrics.

In [10]:
# Import pre-scraped (lyrics) CSV file
lyrics_df = pd.read_csv('songs_genre_lyrics.csv')

# Check how many rows have empty or missing lyrics
empty_lyrics = lyrics_df[lyrics_df['lyrics'].isna() | (lyrics_df['lyrics'].str.strip() == '')]
print(f"Number of rows with empty lyrics: {len(empty_lyrics)}")

# Remove rows with empty or missing lyrics
lyrics_df = lyrics_df[~(lyrics_df['lyrics'].isna() | (lyrics_df['lyrics'].str.strip() == ''))]

lyrics_df

Number of rows with empty lyrics: 671


,Song,Artist,Genre Tags,Tag Source,lyrics
1,Smalltown Boy,Bronski Beat,"80s, new wave, synthpop, pop, synth pop",track,To your soul\nTo your soul\nCry\nCry\nCry\n\nY...
2,I'm Still Standing,Elton John,"pop, 80s, elton john, rock, classic rock",track,You could never know what it's like\nYour bloo...
3,Funky Town,Lipps Inc.,"Disco, 80s, pop, 70s, dance",track,Gotta make a move to a town that's right for m...
4,I'm So Excited,The Pointer Sisters,"80s, Disco, pop, dance, soul",track,Tonight's the night we're gonna make it happen...
5,Cheri Cheri Lady,Modern Talking,"80s, Disco, pop, Modern Talking, dance",track,"Oh, I cannot explain\nEvery time, it's the sam..."
...,...,...,...,...,...
5460,Rock with You - Single Version,Michael Jackson,"pop, michael jackson, 80s, Disco, dance",track,"Girl, close your eyes\nLet that rhythm get int..."
5462,You Sexy Thing,Hot Chocolate,"Disco, 70s, funk, soul, pop",track,I believe in miracles\nWhere're you from?\nYou...
5464,Get It On,T. Rex,"glam rock, 70s, classic rock, rock, glam",track,"Well, you're dirty and sweet\nClad in black, d..."
5466,Love Really Hurts Without You,Billy Ocean,"80s, pop, soul, 70s, Disco",track,You run around town like a fool and you think ...


Dropping rows whose lyrics are not English.

In [11]:
#setting the langdetect seed for reproducibility
DetectorFactory.seed = 0

# Uses langdetect to identify the language of text
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'error'

# Adds a new column 'language' to the dataframe and applies the detect_language function
# Each row has a detected language code; 'en' - English, 'es' - Spanish, 'fr' - French, etc. 
lyrics_df['language'] = lyrics_df['lyrics'].apply(detect_language)

# Filters out non-English songs and removes them from the dataframe.
non_english_df = lyrics_df[lyrics_df['language'] != 'en']
lyrics_df = lyrics_df[~lyrics_df['Song'].isin(non_english_df['Song'])].reset_index(drop=True)

# Resets the index of the dataframe.
lyrics_df = lyrics_df.reset_index(drop=True)

lyrics_df.to_csv('lyrics_df')

Combine the genre and lyric dataframes with the metadata dataframe for a final dataset.

In [12]:
master_df = pd.read_csv('initial_songs_master.csv')

# Merge Genre Tags and lyrics from lyrics_df
# We'll assume the columns to join on are 'artist' and 'title'
columns_to_add = ['Artist', 'Song', 'Genre Tags', 'lyrics']  # adjust names if needed

# Ensure lyrics_df has only the necessary columns
df_to_merge = lyrics_df[columns_to_add]

# Perform the merge
merged_df = master_df.merge(df_to_merge, on=['Artist', 'Song'], how='left')


# Remove rows where lyrics are NaN or just whitespace
final_df = merged_df[~(merged_df['lyrics'].isna() | (merged_df['lyrics'].str.strip() == ''))]

# Reset index for cleanliness
final_df = final_df.reset_index(drop=True)


final_df = final_df.drop(columns=['Genres', 'Parent Genres', 'Time Signature'])

In [13]:
final_df.to_csv('dataset.csv')

In [14]:
df = pd.read_csv('dataset.csv')
df.head()
df.info()
df.describe(include='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4489 entries, 0 to 4488
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    4489 non-null   int64 
 1   Song          4489 non-null   object
 2   Artist        4489 non-null   object
 3   Popularity    4489 non-null   int64 
 4   BPM           4489 non-null   int64 
 5   Time          4125 non-null   object
 6   Dance         4489 non-null   int64 
 7   Energy        4489 non-null   int64 
 8   Acoustic      4489 non-null   int64 
 9   Instrumental  4489 non-null   int64 
 10  Happy         4489 non-null   int64 
 11  Speech        4489 non-null   int64 
 12  Live          4489 non-null   int64 
 13  Loud (Db)     4489 non-null   int64 
 14  Key           4489 non-null   object
 15  Camelot       4489 non-null   object
 16  Artist_Clean  4489 non-null   object
 17  Song_Clean    4489 non-null   object
 18  Genre Tags    4489 non-null   object
 19  lyrics

,Song,Artist,Time,Key,Camelot,Artist_Clean,Song_Clean,Genre Tags,lyrics
count,4489,4489,4125,4489,4489,4489,4489,4489,4489
unique,4316,2009,356,24,24,1632,4284,4180,4471
top,Heaven,Taylor Swift,03:29,C,8B,Drake,Closer,Bryson Tiller,"On my way, on my way\nOn my way, on my way\n\n..."
freq,4,49,45,430,430,66,4,12,2


Note the duplicate songs and lyrics. We will clean that up next. Also note that Key and Camelot have the same amount of unique values.

# Cleaning Duplicate Lyrics and Song Titles.

In [15]:
# Step 1: Sort by popularity descending
df = df.sort_values(by='Popularity', ascending=False)

# Step 2: Drop duplicate lyrics, keeping the most popular version
df = df.drop_duplicates(subset='lyrics', keep='first').reset_index(drop=True)

# Step 3: Find songs with duplicate titles (after lyrics deduplication)
duplicates = (
    df[df['Song'].duplicated(keep=False)]
    [['Song', 'Artist']]
    .sort_values(by='Song')
)

# Step 4: Display in scrollable HTML box
html_box = f"""
<div style="height: 300px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9">
    <h4 style="margin-top:0;">Songs with Duplicate Titles</h4>
    {duplicates.to_html(index=False)}
</div>
"""

display(HTML(html_box))


Song,Artist
23,"Mike WiLL Made-It,Miley Cyrus,Wiz Khalifa,Juicy J"
23,Wallice
23,NLE Choppa
Afraid,Nico
Afraid,The Neighbourhood
After Hours,The Weeknd
After Hours,The Velvet Underground
Ain't No Sunshine,Bill Withers
Ain't No Sunshine,Michael Jackson
Alive,"Dabin,RUNN"


In [16]:
df.describe(include='object')

,Song,Artist,Time,Key,Camelot,Artist_Clean,Song_Clean,Genre Tags,lyrics
count,4471,4471,4109,4471,4471,4471,4471,4471,4471
unique,4299,2008,356,24,24,1632,4271,4175,4471
top,Alive,Taylor Swift,03:29,C,8B,Drake,Alive,Bryson Tiller,(But I wanna stay)\n\nI want you to stay\n'Til...
freq,4,49,45,429,429,66,4,12,1


The difference between count lyrics and unique lyrics is now 0.

The difference between count song and unique song is now much less (172) likely due to common song names.

In [17]:
# Group by the cleaned song name
grouped = df.groupby('Song_Clean')['Song'].nunique()

# Filter where more than one distinct 'song' maps to a single 'song_clean'
duplicates = grouped[grouped > 1]

# Now view those entries
df[df['Song_Clean'].isin(duplicates.index)][['Song_Clean', 'Song', 'Artist', 'Popularity']].sort_values(by='Song_Clean')

,Song_Clean,Song,Artist,Popularity
2527,4 AM,4 AM,"2 Chainz,Travis Scott",62
3268,4 AM,4 AM - Adam K & Soha Radio Edit,Kaskade,51
274,Another One Bites The Dust,Another One Bites The Dust - Remastered 2011,Queen,84
4073,Another One Bites The Dust,Another One Bites The Dust,Queen,1
2625,Babooshka,Babooshka,Kate Bush,61
2311,Babooshka,Babooshka - 2018 Remaster,Kate Bush,65
2209,Boys,Boys - Summertime Love,Sabrina,66
4138,Boys,Boys,Hippo Campus,0
637,Dancing In The Flames,Dancing In The Flames,The Weeknd,80
3216,Dancing In The Flames,Dancing In The Flames - Acoustic,The Weeknd,52


In [18]:
# Step 1: Sort by popularity descending
df = df.sort_values(by='Popularity', ascending=False)

# Step 2: Drop duplicates based on Song_Clean and Artist
# This keeps the most popular version for each (Song_Clean, Artist) pair
df_deduped = df.drop_duplicates(subset=['Song_Clean', 'Artist'], keep='first').reset_index(drop=True)

removed = pd.merge(df, df_deduped, how='outer', indicator=True)
removed = removed[removed['_merge'] == 'left_only']

# View removed entries
removed[['Song_Clean', 'Song', 'Artist', 'Popularity']].sort_values(by='Song_Clean')

,Song_Clean,Song,Artist,Popularity
4408,Another One Bites The Dust,Another One Bites The Dust,Queen,1
1314,Babooshka,Babooshka,Kate Bush,61
4008,Dancing In The Flames,Dancing In The Flames - Acoustic,The Weeknd,52
888,Every Little Thing She Does Is Magic,Every Little Thing She Does Is Magic - Remaste...,The Police,2
1934,Hot Stuff,Hot Stuff - 12 Version,Donna Summer,66
2858,I'm Not Alone,I'm Not Alone - 2009 Remaster,Calvin Harris,50
724,Lola,Lola - 2020 Stereo Remaster,The Kinks,2
641,Message In A Bottle,Message In A Bottle - Remastered 2003,The Police,0
142,Mirrors,Mirrors - Radio Edit,Justin Timberlake,64
247,On The Floor,On The Floor - Radio Edit,"Jennifer Lopez,Pitbull",78


Upon further investigation, Key and Camelot seem to be the exact same thing just in different notation. Will run Cramer's V categorical correlation to confirm this, and if score is one will remove Key from the dataset

In [19]:
import numpy as np
from scipy.stats import chi2_contingency

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))  # bias correction
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2_corr / min((k_corr-1), (r_corr-1)))

cramers_v(df['Key'], df['Camelot'])

# Dropping 'Key' column
df = df.drop(columns=['Key'])

#Remove songs without time
df = df[df['Time'].notnull()]
df = df.reset_index(drop=True)


In [20]:
df.describe()

,Unnamed: 0,Popularity,BPM,Dance,Energy,Acoustic,Instrumental,Happy,Speech,Live,Loud (Db)
count,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000,4109.000000
mean,2415.318326,55.970309,121.808226,58.770260,64.348017,23.628863,4.216598,52.359942,2.733025,13.640789,-7.642979
std,1209.713314,26.733076,27.474388,14.840298,20.016864,26.470157,14.402065,24.407929,7.311542,14.501904,3.438543
min,0.000000,0.000000,49.000000,9.000000,3.000000,0.000000,0.000000,3.000000,0.000000,0.000000,-26.000000
25%,1393.000000,48.000000,101.000000,49.000000,51.000000,2.000000,0.000000,33.000000,0.000000,0.000000,-10.000000
50%,2426.000000,65.000000,121.000000,59.000000,67.000000,12.000000,0.000000,52.000000,0.000000,10.000000,-7.000000
75%,3458.000000,75.000000,138.000000,69.000000,80.000000,38.000000,0.000000,72.000000,0.000000,20.000000,-5.000000
max,4488.000000,100.000000,219.000000,96.000000,100.000000,99.000000,97.000000,99.000000,80.000000,90.000000,-1.000000


In [21]:
df.to_csv('week2_dataset.csv')